In [3]:
!pip install -U "ray[tune]"

  Using cached idna-3.3-py3-none-any.whl (61 kB)


In [ ]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [7]:
def load_data(data_dir="./data"):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

In [9]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
def train_cifar(config, checkpoint_dir=None, data_dir=None):
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
    print("Finished Training")

In [12]:
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [13]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=10, max_num_epochs=10, gpus_per_trial=0)

1.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

7.6%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

14.2%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

20.7%IOPub m

== Status ==
Current time: 2022-03-11 21:44:31 (running for 00:00:00.19)
Memory usage on this node: 15.7/31.9 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/16 CPUs, 0/1 GPUs, 0.0/10.23 GiB heap, 0.0/5.12 GiB objects
Result logdir: C:\Users\user\ray_results\DEFAULT_2022-03-11_21-44-31
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-----------------+--------------+------+------+-------------+
| Trial name          | status   | loc             |   batch_size |   l1 |   l2 |          lr |
|---------------------+----------+-----------------+--------------+------+------+-------------|
| DEFAULT_ff0e0_00000 | RUNNING  | 127.0.0.1:28404 |            2 |  128 |    4 | 0.0136978   |
| DEFAULT_ff0e0_00001 | PENDING  |                 |            2 |  256 |   16 | 0.0666173   |
| DEFAULT_ff0e0_00002 | PENDING  |                 |            8 |   64 |   64 | 0.02

 pid=28404) Traceback (most recent call last):
 pid=28404)   File "<string>", line 1, in <module>
 pid=28404)   File "C:\Users\user\anaconda3\envs\coco\lib\multiprocessing\spawn.py", line 116, in spawn_main
 pid=28404)     exitcode = _main(fd, parent_sentinel)
 pid=28404)   File "C:\Users\user\anaconda3\envs\coco\lib\multiprocessing\spawn.py", line 126, in _main
 pid=28404)     self = reduction.pickle.load(from_parent)
 pid=28404)   File "C:\Users\user\anaconda3\envs\coco\lib\site-packages\torch\__init__.py", line 124, in <module>
 pid=28404)     raise err
 pid=28404) OSError: [WinError 1455] �� �۾��� �Ϸ��ϱ� ���� ����¡ ������ �ʹ� �۽��ϴ�. Error loading "C:\Users\user\anaconda3\envs\coco\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.


 pid=16312) Files already downloaded and verified
 pid=28600) Files already downloaded and verified
 pid=3424) Files already downloaded and verified
 pid=11348) Files already downloaded and verified
 pid=29024) Files already downloaded and verified


 pid=26896) 2022-03-11 21:44:38,740	ERROR function_runner.py:268 -- Runner Thread raised error.
 pid=26896) Traceback (most recent call last):
 pid=26896)   File "C:\Users\user\anaconda3\envs\coco\lib\site-packages\ray\tune\function_runner.py", line 262, in run
 pid=26896)     self._entrypoint()
 pid=26896)   File "C:\Users\user\anaconda3\envs\coco\lib\site-packages\ray\tune\function_runner.py", line 330, in entrypoint
 pid=26896)     return self._trainable_func(self.config, self._status_reporter,
 pid=26896)   File "C:\Users\user\anaconda3\envs\coco\lib\site-packages\ray\util\tracing\tracing_helper.py", line 451, in _resume_span
 pid=26896)     return method(self, *_args, **_kwargs)
 pid=26896)   File "C:\Users\user\anaconda3\envs\coco\lib\site-packages\ray\tune\function_runner.py", line 600, in _trainable_func
 pid=26896)     output = fn()
 pid=26896)   File "C:\Users\user\AppData\Local\Temp\ipykernel_10172\2069882394.py", line 40, in train_cifar
 pid=26896)   File "C:\Users\user\ana

...

2022-03-11 21:44:46,305	WARNING worker.py:1326 -- Traceback (most recent call last):
  File "python\ray\_raylet.pyx", line 633, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 674, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 640, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 644, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 593, in ray._raylet.execute_task.function_executor
  File "C:\Users\user\anaconda3\envs\coco\lib\site-packages\ray\_private\function_manager.py", line 648, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "C:\Users\user\anaconda3\envs\coco\lib\site-packages\ray\_private\function_manager.py", line 544, in temporary_actor_method
    raise RuntimeError(
RuntimeError: The actor with name ImplicitFunc failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:

Traceback (most recent call last):
 

Result for DEFAULT_ff0e0_00009:
  trial_id: ff0e0_00009
  
== Status ==
Current time: 2022-03-11 21:44:47 (running for 00:00:16.23)
Memory usage on this node: 20.0/31.9 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 14.0/16 CPUs, 0/1 GPUs, 0.0/10.23 GiB heap, 0.0/5.12 GiB objects
Result logdir: C:\Users\user\ray_results\DEFAULT_2022-03-11_21-44-31
Number of trials: 10/10 (3 ERROR, 7 RUNNING)
+---------------------+----------+-----------------+--------------+------+------+-------------+
| Trial name          | status   | loc             |   batch_size |   l1 |   l2 |          lr |
|---------------------+----------+-----------------+--------------+------+------+-------------|
| DEFAULT_ff0e0_00000 | RUNNING  | 127.0.0.1:28404 |            2 |  128 |    4 | 0.0136978   |
| DEFAULT_ff0e0_00002 | RUNNING  | 127.0.0.1:25368 |            8 |   64 |   64 | 0.028651    |
| DEFAULT_ff0e0_00003 | RUNN

2022-03-11 22:06:57,010	WARNING tune.py:595 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


== Status ==
Current time: 2022-03-11 22:06:57 (running for 00:22:25.90)
Memory usage on this node: 19.9/31.9 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 14.0/16 CPUs, 0/1 GPUs, 0.0/10.23 GiB heap, 0.0/5.12 GiB objects
Result logdir: C:\Users\user\ray_results\DEFAULT_2022-03-11_21-44-31
Number of trials: 10/10 (3 ERROR, 7 RUNNING)
+---------------------+----------+-----------------+--------------+------+------+-------------+
| Trial name          | status   | loc             |   batch_size |   l1 |   l2 |          lr |
|---------------------+----------+-----------------+--------------+------+------+-------------|
| DEFAULT_ff0e0_00000 | RUNNING  | 127.0.0.1:28404 |            2 |  128 |    4 | 0.0136978   |
| DEFAULT_ff0e0_00002 | RUNNING  | 127.0.0.1:25368 |            8 |   64 |   64 | 0.028651    |
| DEFAULT_ff0e0_00003 | RUNNING  | 127.0.0.1:11348 |            8 |  128 |    4 | 0.005


KeyboardInterrupt

